## Corona-News Scraper from Tagesschau.de

In [14]:
import re
import requests
from bs4 import BeautifulSoup

import datetime
import pandas as pd

In [15]:
todays_date = datetime.datetime.now()
date_stamp = str(todays_date.date())

In [16]:
r = requests.get('https://www.tagesschau.de/')
soup = BeautifulSoup(r.text, "html5lib")


### Parse Tagesschau.de homepage for new articles

In [17]:
headlines = []
dachzeilen = []
teasers = []
links = []
texts = []
a = soup.find_all('a')
for item in a:
    if item.find('p', class_='dachzeile'):
        links.append(item.get('href'))
        texts.append(item.get_text())

In [18]:
# check the first text / link
print(texts[0])

print(links[0])



IW-Chef zu Corona-Krise
"Corona-Bonds sind kein Dauerinstrument"
Die Corona-Krise und ihre wirtschaftlichen Folgen werden unfassbar teuer. Schwer betroffene EU-Länder wie Italien fordern Solidarität - etwa in Form von Corona-Bonds. Auch IW-Chef Hüther sprach sich in den tagesthemen klar dafür aus.
| mehr


/wirtschaft/corona-bonds-101.html


### Check which articles on the homepage are about the Corona virus

In [19]:
# corona, covid19, coronavirus
corona_links = []
corona_texts = []
for i in range(len(texts)):
    if ("corona" in texts[i].lower()) | ("covid" in texts[i].lower()):
        if links[i].startswith("/"):
            corona_links.append(links[i])
            corona_texts.append(texts[i])

In [20]:
print(corona_links)
print("")
print(corona_texts)

['/wirtschaft/corona-bonds-101.html', '/inland/corona-245.html', '/wirtschaft/corona-konjunkturprogramme-101.html', '/inland/corona-bundesregierung-103.html', '/wirtschaft/corona-gutscheine-101.html', '/newsticker/liveblog-corona-sonntag-105.html', '/ausland/coronavirus-spanien-107.html', '/ausland/corona-grossbritannien-starmer-101.html', '/ausland/coronavirus-grossbritannien-103.html', '/ausland/corona-fluechtlingslager-101.html', '/inland/bab-helden-101.html', '/ausland/coronavirus-karten-101.html', '/faktenfinder/faktenchecks-corona-101.html', '/ausland/corona-papst-franziskus-101.html', '/ausland/corona-ramadan-101.html']

['\nIW-Chef zu Corona-Krise\n"Corona-Bonds sind kein Dauerinstrument"\nDie Corona-Krise und ihre wirtschaftlichen Folgen werden unfassbar teuer. Schwer betroffene EU-Länder wie Italien fordern Solidarität - etwa in Form von Corona-Bonds. Auch IW-Chef Hüther sprach sich in den tagesthemen klar dafür aus.\n|\xa0mehr\n\n', '\nForderungen der FDP\nSteuern senken, Ko

### Get full article for each news about corona virus

In [21]:
corona_articles = []
for link in corona_links:
    full_link = "https://www.tagesschau.de" + link
    print(full_link)
    html = urlopen(full_link)
    sub_soup = BeautifulSoup(html, "html5lib")
    article = sub_soup.find_all('p', class_='text small')
    paragraphs = []
    for i in range(len(article)):
        paragraphs.append(article[i].get_text())
    # unifying the paragraphs
    final_article = " ".join(paragraphs)
    corona_articles.append(final_article)
    

https://www.tagesschau.de/wirtschaft/corona-bonds-101.html
https://www.tagesschau.de/inland/corona-245.html
https://www.tagesschau.de/wirtschaft/corona-konjunkturprogramme-101.html
https://www.tagesschau.de/inland/corona-bundesregierung-103.html
https://www.tagesschau.de/wirtschaft/corona-gutscheine-101.html
https://www.tagesschau.de/newsticker/liveblog-corona-sonntag-105.html
https://www.tagesschau.de/ausland/coronavirus-spanien-107.html
https://www.tagesschau.de/ausland/corona-grossbritannien-starmer-101.html
https://www.tagesschau.de/ausland/coronavirus-grossbritannien-103.html
https://www.tagesschau.de/ausland/corona-fluechtlingslager-101.html
https://www.tagesschau.de/inland/bab-helden-101.html
https://www.tagesschau.de/ausland/coronavirus-karten-101.html
https://www.tagesschau.de/faktenfinder/faktenchecks-corona-101.html
https://www.tagesschau.de/ausland/corona-papst-franziskus-101.html
https://www.tagesschau.de/ausland/corona-ramadan-101.html


In [22]:
print(corona_articles)

['Die Corona-Krise und ihre wirtschaftlichen Folgen werden unfassbar teuer. Schwer betroffene EU-Länder wie Italien fordern Solidarität - etwa in Form von Corona-Bonds. Auch IW-Chef Hüther sprach sich in den tagesthemen klar dafür aus.  Im Kampf gegen die wirtschaftlichen Folgen der Corona-Pandemie herrscht derzeit Streit in der EU. So verlangen unter anderem die schwer betroffenen Länder Italien und Spanien gemeinsame Anleihen der EU-Mitglieder zur Finanzierung der EU-Staaten - sogenannte Corona-Bonds. Damit könnten auch bereits hoch verschuldete Länder wie eben Italien Geld zu günstigeren Konditionen einsammeln, weil wirtschaftlich stärkere Staaten wie Deutschland ebenfalls für Zinsen und Rückzahlung haften. Nichts da, sagt dazu die Bundesregierung. Sie treibt die Sorge um, dass Deutschland damit praktisch für die Schulden anderer Länder mit haftet und daraus ein Dauerinstrument wird. Auch Österreich oder die Niederlande sind strikt dagegen. \n  Unterstützung erhalten die Bonds-Befür

In [23]:
len(corona_articles)

15

### Make into CSV file

In [24]:
columns = ["date", "url", "article", "resort"]
df = pd.DataFrame(columns=columns)
df["url"] = corona_links
df["article"] = corona_articles
df["date"] = todays_date
df["resort"] = df["url"].apply(lambda x: x.split("/")[1])

In [25]:
df.head()

,date,url,article,resort
0,2020-04-05 20:57:11.925137,/wirtschaft/corona-bonds-101.html,Die Corona-Krise und ihre wirtschaftlichen Fol...,wirtschaft
1,2020-04-05 20:57:11.925137,/inland/corona-245.html,FDP-Chef Lindner will die wirtschaftlichen Cor...,inland
2,2020-04-05 20:57:11.925137,/wirtschaft/corona-konjunkturprogramme-101.html,"Konjunkturpakete, Steuersenkungen, ein neuer M...",wirtschaft
3,2020-04-05 20:57:11.925137,/inland/corona-bundesregierung-103.html,Nach Ansicht der Regierung steht Deutschland d...,inland
4,2020-04-05 20:57:11.925137,/wirtschaft/corona-gutscheine-101.html,Gutscheine statt Geld zurück - diese Devise ha...,wirtschaft


### Export CSV

In [26]:
df.to_csv("data/" + date_stamp)